In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
import xarray as xr

In [59]:
#Replace with file path for this directory on your machine
!pip install -e /Users/justinmaynard/Documents/GitHub/assetraMP/ 

Obtaining file:///Users/justinmaynard/Documents/GitHub/assetraMP
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for assetra (pyproject.toml) ... done
  Created wheel for assetra: filename=assetra-1.0.3-py3-none-any.whl size=2685 sha256=18cd4e6751436938adbaafd86485445c9a61669e98076973087c43a68b8a5bf0
  Stored in directory: /private/var/folders/zd/3zqsbnbn5lg29wjyl0df33l80000gn/T/pip-ephem-wheel-cache-sz95789w/wheels/c0/f5/c9/3b09bf7f2c6beadc5d6cde6ca7e50094804a4b90493c211e80
Successfully built assetra
  Attempting uninstall: assetra
    Found existing installation: assetra 1.0.3
    Uninstalling assetra-1.0.3:
      Successfully uninstalled assetra-1.0.3


In [60]:
import sys
sys.path.append('/Users/justinmaynard/Documents/GitHub/assetraMP/')
import assetra 

In [ ]:
p = Path(".")
root_folder = p.cwd().parent
root_folder
data_folder = root_folder / 'pjm_2023_test' / 'pjm_data'
data_folder
scripts_foler = root_folder/ 'pjm_2023_test'

In [ ]:
from assetra.system import EnergySystem
from assetra.system import EnergySystemBuilder

builder = EnergySystemBuilder()
unit_count = 0

### Demand Data

In [14]:
#Function to load demand data
def load_pjm_cleaned_hourly_demand(
        pjm_demand_file: Path,
        start_hour: datetime,
        end_hour: datetime) -> xr.DataArray:
    """Return hourly demand data as formatted data array.
    To use this function, download cleaned demand data from:

    https://github.com/truggles/EIA_Cleaned_Hourly_Electricity_Demand_Data

    Args:
        eia_930_cleaned_demand_file (Path): Path to hourly demand file
        start_hour (datetime): First timestamp to include
        end_hour (datetime): Last timestamp to include (inclusive)

    Returns:
        xr.DataArray: Hourly demand array with time dimension and datetime coordinates.
    """
    # read demand file
    pjm_demand_df = pd.read_csv(
        pjm_demand_file,
        usecols=["datetime_beginning_ept", "mw"],
        index_col="datetime_beginning_ept",
        parse_dates=True,
    )

    # keep cleaned demand demand
    pjm_hourly_demand_pd = pjm_demand_df["mw"].loc[start_hour:end_hour]

    # convert to xr.DataArray
    pjm_hourly_demand = xr.DataArray(
        data=pjm_hourly_demand_pd.values,
        coords=dict(
            time=pjm_hourly_demand_pd.index.values
        )
    )
    return pjm_hourly_demand

In [23]:
#relative, should work for all
demand_path = str(data_folder/"hrl_load_metered.csv")

def sum_by_datetime(file_path):
    # Load the CSV file
    data = pd.read_csv(file_path)
    # Ensure datetime column is parsed correctly
    data['datetime_beginning_ept'] = pd.to_datetime(data['datetime_beginning_ept'])
    # Group by 'datetime_beginning_ept' and sum 'mw'
    grouped_data = data.groupby('datetime_beginning_ept', as_index=False)['mw'].sum()
    grouped_data.index = pd.to_datetime(grouped_data['datetime_beginning_ept'])
    grouped_data = grouped_data.drop(columns='datetime_beginning_ept')

    return grouped_data

# Apply the function to the uploaded file

summed_data = sum_by_datetime(demand_path)


/var/folders/zd/3zqsbnbn5lg29wjyl0df33l80000gn/T/ipykernel_39293/3196234918.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['datetime_beginning_ept'] = pd.to_datetime(data['datetime_beginning_ept'])


In [24]:
summed_data

,mw
datetime_beginning_ept,
2023-01-01 00:00:00,146546.274
2023-01-01 01:00:00,142981.658
2023-01-01 02:00:00,140047.470
2023-01-01 03:00:00,138552.566
2023-01-01 04:00:00,138456.840
...,...
2024-01-01 19:00:00,194766.828
2024-01-01 20:00:00,191178.494
2024-01-01 21:00:00,187060.136


In [25]:
summed_data.to_csv(str(data_folder/"pjm_load_cleaned.csv"))

In [27]:
#Load demand data
pjm_cleaned_demand_file = Path( data_folder / "pjm_load_cleaned.csv")
hourly_demand = load_pjm_cleaned_hourly_demand(
	pjm_cleaned_demand_file,
	start_hour="2023-01-01 00:00:00",
	end_hour="2023-12-31 23:00:00"
)


In [30]:
# create demand unit
from assetra.units import DemandUnit

builder.add_unit(
    DemandUnit(
        id=unit_count,
        hourly_demand=hourly_demand
    )
)
unit_count += 1

### EIA 860 Data

In [31]:
def load_eia_860_plants(eia_860_plant_file: Path, bal_auth: str) -> pd.DataFrame:
    """Return a subset of the EIA 860 plant file for plants in a balancing authority

    Args:
        eia_860_plant_file (Path): Path to hourly demand file
        bal_auth (str): Balancing authority code as defined by EIA-860

    Returns:
        pd.DataFrame: Plant code-indexed dataframe with plant latitude and longitude
    """
    # read file
    eia_860_plant_df = pd.read_excel(
        eia_860_plant_file,
        skiprows=1,
        usecols=[
            "Plant Code",
            "Latitude",
            "Longitude",
            "Balancing Authority Code",
        ],
        index_col="Plant Code",
    )

    # filter
    eia_860_plant_df = eia_860_plant_df[
        eia_860_plant_df["Balancing Authority Code"] == bal_auth
    ]
    return eia_860_plant_df

In [32]:
# parse eia 860 plants (selecting by balancing authority)
eia_860_plant_file = Path( data_folder / "2___Plant_Y2023.xlsx")
eia_860_plants = load_eia_860_plants(eia_860_plant_file, "PJM")

In [33]:
def load_eia_860_generators(
    eia_860_generator_file: Path,
    eia_860_plants: pd.DataFrame,
    additional_cols: list=[],
    tech_filter: list=[],
    invert_tech_filter: bool=False
    ) -> pd.DataFrame:
    """Return dataframe with generators" latitude, longitude, technology, and nameplate capacity"""
    # read file
    eia_860_generator_df = pd.read_excel(
        eia_860_generator_file,
        skiprows=1,
        usecols=[
            "Plant Code",
            "Technology",
            "Nameplate Capacity (MW)",
            "Status"
        ] + additional_cols,
    )

    # filter by plants
    eia_860_generator_df = eia_860_generator_df[
        eia_860_generator_df["Plant Code"].isin(eia_860_plants.index)
    ]

    # filter by technology
    if tech_filter:
        if invert_tech_filter:
            eia_860_generator_df = eia_860_generator_df[
                ~eia_860_generator_df["Technology"].isin(
                    tech_filter
                )
            ]
        else:
            eia_860_generator_df = eia_860_generator_df[
                eia_860_generator_df["Technology"].isin(
                    tech_filter
                )
            ]

    # filter by status
    eia_860_generator_df = eia_860_generator_df[
        eia_860_generator_df["Status"] == "OP"
    ]

    eia_860_generator_df["Latitude"] = eia_860_generator_df["Plant Code"].map(lambda plant_code: eia_860_plants["Latitude"][plant_code])
    eia_860_generator_df["Longitude"] = eia_860_generator_df["Plant Code"].map(lambda plant_code: eia_860_plants["Longitude"][plant_code])

    return eia_860_generator_df

In [34]:
# parse eia 860 generator types
EIA_860_NON_THERMAL_TECHNOLOGY = [
    "Onshore Wind Turbine",
    #"Conventional Hydroelectric",
    "Solar Photovoltaic",
    "Offshore Wind Turbine",
    "Batteries",
    "Hydroelectric Pumped Storage"
]
eia_860_generator_file = Path(data_folder / "3_1_Generator_Y2023.xlsx")
eia_860_wind_file = Path(data_folder / "3_2_Wind_Y2023.xlsx")
eia_860_solar_file = Path(data_folder / "3_3_Solar_Y2023.xlsx")
eia_860_storage_file = Path(data_folder / "3_4_Energy_Storage_Y2023.xlsx")

eia_860_thermal_generators = load_eia_860_generators(
    eia_860_generator_file, 
    eia_860_plants,
    tech_filter=EIA_860_NON_THERMAL_TECHNOLOGY,
    invert_tech_filter=True
)
eia_860_wind_generators = load_eia_860_generators(
    eia_860_wind_file,
    eia_860_plants
)
eia_860_solar_generators = load_eia_860_generators(
    eia_860_solar_file,
    eia_860_plants
)
eia_860_storage_generators = load_eia_860_generators(
    eia_860_storage_file,
    eia_860_plants,
    additional_cols=["Nameplate Energy Capacity (MWh)"]
)

### Import MERRA

In [35]:
import xarray as xr

# load processed power generation dataset (solar cf, wind cf, and temperature)
pow_gen_file = Path(data_folder / "pjm_power_generation.nc")
pow_gen_dataset = xr.open_dataset(pow_gen_file)

def get_nearest_hourly_profile(
    latitude: float,
    longitude: float,
    array: xr.DataArray
) -> xr.DataArray:
    """Return time series corresponding to the nearest coordinate in a
    MERRA power generation data array.

    Args:
        latitude (float): Latitude relative to equator in degrees
        start_hour (datetime): Longitude relative to meridian in degrees
        array (xr.DataArray): "solar_capacity_factor", "wind_capacity_factor",
            or "temperature"

    Returns:
        xr.DataArray: Array with time dimension and datetime coordinates.
    """
    return array.sel(
            lat=latitude, 
            lon=longitude, 
            method="nearest"
        ).squeeze(drop=True)

def get_merra_power_generation_solar_cf(
    latitude: float,
    longitude: float) -> xr.DataArray:
    return get_nearest_hourly_profile(latitude, longitude, pow_gen_dataset["solar_capacity_factor"])

def get_merra_power_generation_wind_cf(
    latitude: float,
    longitude: float) -> xr.DataArray:
    return get_nearest_hourly_profile(latitude, longitude, pow_gen_dataset["wind_capacity_factor"])

def get_merra_power_generation_temperature(
    latitude: float,
    longitude: float) -> xr.DataArray:
    return get_nearest_hourly_profile(latitude, longitude, pow_gen_dataset["temperature"])

In [36]:
import pandas as pd

# load temperature dependent outage rate (tdfor) table
tdfor_table_file = Path(data_folder / "temperature_dependent_outage_rates.csv")
tdfor_table = pd.read_csv(tdfor_table_file, index_col=0)
tdfor_table = tdfor_table / 100 # percentages stored as integers

# create mapping table for tdfor table
tech_categories = {
    "CC" : ["Natural Gas Fired Combined Cycle"],
    "CT" : ["Natural Gas Fired Combustion Turbine","Landfill Gas"],
    "DS" : ["Natural Gas Internal Combustion Engine"],
    "ST" : ["Conventional Steam Coal","Natural Gas Steam Turbine"],
    "NU" : ["Nuclear"],
    "HD" : ["Conventional Hydroelectric","Solar Thermal without Energy Storage",
                   "Hydroelectric Pumped Storage","Solar Thermal with Energy Storage","Wood/Wood Waste Biomass"]
}

# create mapping from technology to category
tech_mapping = {tech : cat for cat, techs in tech_categories.items() for tech in techs}

def get_hourly_forced_outage_rate(hourly_temperature: xr.DataArray, technology: str) -> xr.DataArray:
    # index tdfor table by tech
    tdfor_map = tdfor_table[tech_mapping.get(technology, "Other")]
    map_temp_to_for = lambda hourly_temperature: tdfor_map.iloc[
            tdfor_map.index.get_indexer(hourly_temperature, method="nearest")
        ]
    return xr.apply_ufunc(
        map_temp_to_for,
        hourly_temperature
    ).rename("hourly_forced_outage_rate")


### Build Units

In [37]:
from assetra.units import StochasticUnit

for _, generator in eia_860_thermal_generators.iterrows():
    # get hourly temperature
    hourly_temperature = get_merra_power_generation_temperature(
        generator["Latitude"],
        generator["Longitude"]
    )

    # map temperature to hourly forced outage rate
    hourly_forced_outage_rate = get_hourly_forced_outage_rate(hourly_temperature, generator["Technology"])

    # get hourly capacity
    hourly_capacity = ( 
        xr.ones_like(hourly_temperature).rename("hourly_capacity") 
        * generator["Nameplate Capacity (MW)"]
    )

    # create assetra energy unit
    thermal_unit = StochasticUnit(
            id=unit_count,
            nameplate_capacity=generator["Nameplate Capacity (MW)"],
            hourly_capacity=hourly_capacity,
            hourly_forced_outage_rate=hourly_forced_outage_rate
        )
    unit_count += 1
    
    # add unit to energy system
    builder.add_unit(thermal_unit)

In [38]:


from assetra.units import StochasticUnit

# add solar
for _, generator in eia_860_solar_generators.iterrows():
    # get hourly temperature
    hourly_temperature = get_merra_power_generation_temperature(
        generator["Latitude"],
        generator["Longitude"]
    )
    # get hourly temperature
    hourly_capacity = get_merra_power_generation_solar_cf(
        generator["Latitude"],
        generator["Longitude"]
    ) * generator["Nameplate Capacity (MW)"]

    # map temperature to hourly forced outage rate
    hourly_forced_outage_rate = get_hourly_forced_outage_rate(hourly_temperature, generator["Technology"])

    # create assetra energy unit
    solar_unit = StochasticUnit(
            id=unit_count,
            nameplate_capacity=generator["Nameplate Capacity (MW)"],
            hourly_capacity=hourly_capacity,
            hourly_forced_outage_rate=hourly_forced_outage_rate
        )
    unit_count += 1
    
    # add unit to energy system
    builder.add_unit(solar_unit)

# add wind
for _, generator in eia_860_wind_generators.iterrows():
    # get hourly temperature
    hourly_temperature = get_merra_power_generation_temperature(
        generator["Latitude"],
        generator["Longitude"]
    )
    # get hourly temperature
    hourly_capacity = get_merra_power_generation_wind_cf(
        generator["Latitude"],
        generator["Longitude"]
    ) * generator["Nameplate Capacity (MW)"]

    # map temperature to hourly forced outage rate
    hourly_forced_outage_rate = get_hourly_forced_outage_rate(hourly_temperature, generator["Technology"])

    # create assetra energy unit
    wind_unit = StochasticUnit(
            id=unit_count,
            nameplate_capacity=generator["Nameplate Capacity (MW)"],
            hourly_capacity=hourly_capacity,
            hourly_forced_outage_rate=hourly_forced_outage_rate
        )
    unit_count += 1
    
    # add unit to energy system
    builder.add_unit(wind_unit)



In [39]:
from assetra.units import StorageUnit

STORAGE_EFFICIENCY = 0.8

for _, generator in eia_860_storage_generators.iterrows():
    storage_unit = StorageUnit(
        id=unit_count,
        nameplate_capacity=generator["Nameplate Capacity (MW)"],
        charge_rate=generator["Nameplate Capacity (MW)"],
        discharge_rate=generator["Nameplate Capacity (MW)"],
        charge_capacity=generator["Nameplate Energy Capacity (MWh)"],
        roundtrip_efficiency=STORAGE_EFFICIENCY
    )
    unit_count += 1

    # add unit to energy system
    builder.add_unit(storage_unit)

In [42]:

pjm_system_dir = Path(data_folder / "pjm_energy_system")

if pjm_system_dir.exists():
    energy_system = EnergySystem()
    energy_system.load(pjm_system_dir)
else:
    print("PJM saved system not found. Please create and save this system following the instructions found in the appendix") 

In [41]:
energy_system = builder.build()
energy_system.save(pjm_system_dir)

In [43]:
if pjm_system_dir.exists():
    energy_system = EnergySystem()
    energy_system.load(pjm_system_dir)
else:
    print("PJM saved system not found. Please create and save this system following the instructions found in the appendix") 

In [44]:
print("# of Units:", energy_system.size)
print("Sys. Capacity (MW):", round(energy_system.system_capacity))

# of Units: 3466
Sys. Capacity (MW): 212271


In [47]:
demandUnit = xr.open_dataset(data_folder/ "pjm_energy_system/DemandUnit.assetra.nc")
demandUnit.to_dataframe() #.to_csv("demandunit.csv")

nameplate_capacity  hourly_capacity
energy_unit time                                                    
0           2023-01-01 00:00:00                   0      -146546.274
            2023-01-01 01:00:00                   0      -142981.658
            2023-01-01 02:00:00                   0      -140047.470
            2023-01-01 03:00:00                   0      -138552.566
            2023-01-01 04:00:00                   0      -138456.840
...                                             ...              ...
            2023-12-31 19:00:00                   0      -186265.690
            2023-12-31 20:00:00                   0      -182618.564
            2023-12-31 21:00:00                   0      -178170.472
            2023-12-31 22:00:00                   0      -173316.546
            2023-12-31 23:00:00                   0      -168846.804

[8759 rows x 2 columns]

In [62]:
from assetra.simulation import ProbabilisticSimulation

simulation = ProbabilisticSimulation(
    start_hour="2023-01-01 00:00:00",
    end_hour="2023-12-31 23:00:00",
    trial_size=1
)

simulation.assign_energy_system(energy_system)
simulation.run()

KeyError: "not all values found in index 'time'"